In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [3]:
training_labels = pd.read_csv("set_entrenamiento.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [4]:
training_labels = training_labels[['person','label']]
training_labels['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [5]:
data.head()

,person,Direct,Email,Organic,Paid,Referral,Social,ad campaign hit,brand listing,checkout,...,minimo,suma_resolution,promedio_resolution,maximo_resolution,minimo_resolution,Unnamed: 0,promedio de TFIdf modelo,Suma de TFidf modelo,Maximo de TFIdf modelo,Minimo de TFIdf modelo
0,0008ed71,2.0,0.0,0.0,0.0,0.0,0.0,0,0,3,...,20180517122747,4147200,691200.000000,2073600,0,0.0,1.617163,4.851488,1.859879,1.300103
1,00091926,1.0,0.0,25.0,0.0,0.0,0.0,15,25,2,...,20180503220829,26738688,59684.571429,786432,0,1.0,1.490856,557.580165,2.209395,1.000000
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1,5,0,...,20180326145111,230400,23040.000000,230400,0,2.0,1.219303,3.657909,1.357806,1.000000
3,000ba417,0.0,0.0,6.0,0.0,0.0,0.0,1,24,6,...,20180517111145,4718592,22905.786408,786432,0,3.0,1.798317,287.730661,2.299973,1.000000
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,20180529002747,230400,13552.941176,230400,0,4.0,1.000000,4.000000,1.000000,1.000000


In [6]:
features = pd.merge(training_labels, data, on='person', how='inner')
features = features.fillna(0)
features['label'].value_counts()

1    980
0    980
Name: label, dtype: int64

In [7]:
labels = features['label']
features = features.drop(['label','person'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  1470 Test:  490


In [8]:
import catboost as cb
cat_features_index = [0,1,2,3,4,5,6]

In [9]:
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

In [10]:
import catboost as cb
from catboost import CatBoostClassifier

params = {'depth': [4, 7, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [300]}
cb = CatBoostClassifier()
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
cb_model.fit(x_train, y_train)

0:	learn: 0.6811537	total: 67.4ms	remaining: 20.1s
1:	learn: 0.6703922	total: 82.8ms	remaining: 12.3s
2:	learn: 0.6595103	total: 91.6ms	remaining: 9.06s
3:	learn: 0.6506584	total: 98.7ms	remaining: 7.3s
4:	learn: 0.6412822	total: 105ms	remaining: 6.18s
5:	learn: 0.6329984	total: 110ms	remaining: 5.4s
6:	learn: 0.6258714	total: 116ms	remaining: 4.86s
7:	learn: 0.6181244	total: 121ms	remaining: 4.43s
8:	learn: 0.6113467	total: 127ms	remaining: 4.1s
9:	learn: 0.6020621	total: 132ms	remaining: 3.84s
10:	learn: 0.5957464	total: 138ms	remaining: 3.63s
11:	learn: 0.5881537	total: 144ms	remaining: 3.45s
12:	learn: 0.5819838	total: 150ms	remaining: 3.31s
13:	learn: 0.5797757	total: 155ms	remaining: 3.17s
14:	learn: 0.5724247	total: 161ms	remaining: 3.05s
15:	learn: 0.5663613	total: 166ms	remaining: 2.95s
16:	learn: 0.5633113	total: 172ms	remaining: 2.86s
17:	learn: 0.5586334	total: 177ms	remaining: 2.78s
18:	learn: 0.5526819	total: 187ms	remaining: 2.76s
19:	learn: 0.5485976	total: 194ms	remain

170:	learn: 0.4133155	total: 1.07s	remaining: 807ms
171:	learn: 0.4127583	total: 1.08s	remaining: 802ms
172:	learn: 0.4123721	total: 1.08s	remaining: 796ms
173:	learn: 0.4122124	total: 1.09s	remaining: 789ms
174:	learn: 0.4117971	total: 1.09s	remaining: 783ms
175:	learn: 0.4114429	total: 1.1s	remaining: 778ms
176:	learn: 0.4111740	total: 1.11s	remaining: 771ms
177:	learn: 0.4109597	total: 1.11s	remaining: 764ms
178:	learn: 0.4106233	total: 1.12s	remaining: 758ms
179:	learn: 0.4101938	total: 1.13s	remaining: 752ms
180:	learn: 0.4099603	total: 1.14s	remaining: 746ms
181:	learn: 0.4092347	total: 1.14s	remaining: 740ms
182:	learn: 0.4091228	total: 1.15s	remaining: 734ms
183:	learn: 0.4084065	total: 1.15s	remaining: 727ms
184:	learn: 0.4079471	total: 1.16s	remaining: 720ms
185:	learn: 0.4076989	total: 1.16s	remaining: 714ms
186:	learn: 0.4075223	total: 1.17s	remaining: 707ms
187:	learn: 0.4069955	total: 1.18s	remaining: 700ms
188:	learn: 0.4063734	total: 1.18s	remaining: 694ms
189:	learn: 0

34:	learn: 0.5258212	total: 204ms	remaining: 1.54s
35:	learn: 0.5230869	total: 210ms	remaining: 1.54s
36:	learn: 0.5217372	total: 217ms	remaining: 1.54s
37:	learn: 0.5200732	total: 225ms	remaining: 1.55s
38:	learn: 0.5188563	total: 234ms	remaining: 1.57s
39:	learn: 0.5176105	total: 240ms	remaining: 1.56s
40:	learn: 0.5168966	total: 245ms	remaining: 1.55s
41:	learn: 0.5150003	total: 251ms	remaining: 1.54s
42:	learn: 0.5124550	total: 256ms	remaining: 1.53s
43:	learn: 0.5096542	total: 262ms	remaining: 1.52s
44:	learn: 0.5085603	total: 267ms	remaining: 1.51s
45:	learn: 0.5073796	total: 273ms	remaining: 1.51s
46:	learn: 0.5066375	total: 279ms	remaining: 1.5s
47:	learn: 0.5051970	total: 284ms	remaining: 1.49s
48:	learn: 0.5042776	total: 290ms	remaining: 1.48s
49:	learn: 0.5032366	total: 295ms	remaining: 1.48s
50:	learn: 0.5006591	total: 301ms	remaining: 1.47s
51:	learn: 0.4999088	total: 306ms	remaining: 1.46s
52:	learn: 0.4982698	total: 311ms	remaining: 1.45s
53:	learn: 0.4963129	total: 317m

210:	learn: 0.4217045	total: 1.2s	remaining: 506ms
211:	learn: 0.4214502	total: 1.2s	remaining: 500ms
212:	learn: 0.4213829	total: 1.21s	remaining: 495ms
213:	learn: 0.4212816	total: 1.22s	remaining: 490ms
214:	learn: 0.4211363	total: 1.23s	remaining: 485ms
215:	learn: 0.4208713	total: 1.23s	remaining: 479ms
216:	learn: 0.4207319	total: 1.24s	remaining: 473ms
217:	learn: 0.4202356	total: 1.24s	remaining: 467ms
218:	learn: 0.4199732	total: 1.25s	remaining: 461ms
219:	learn: 0.4194782	total: 1.25s	remaining: 455ms
220:	learn: 0.4188527	total: 1.26s	remaining: 450ms
221:	learn: 0.4183963	total: 1.26s	remaining: 444ms
222:	learn: 0.4178206	total: 1.27s	remaining: 438ms
223:	learn: 0.4174824	total: 1.27s	remaining: 432ms
224:	learn: 0.4170196	total: 1.28s	remaining: 427ms
225:	learn: 0.4164896	total: 1.28s	remaining: 421ms
226:	learn: 0.4159708	total: 1.29s	remaining: 415ms
227:	learn: 0.4157300	total: 1.29s	remaining: 409ms
228:	learn: 0.4153750	total: 1.3s	remaining: 403ms
229:	learn: 0.4

70:	learn: 0.4909673	total: 406ms	remaining: 1.31s
71:	learn: 0.4903513	total: 413ms	remaining: 1.31s
72:	learn: 0.4885176	total: 420ms	remaining: 1.3s
73:	learn: 0.4871270	total: 426ms	remaining: 1.3s
74:	learn: 0.4859556	total: 435ms	remaining: 1.3s
75:	learn: 0.4854899	total: 440ms	remaining: 1.3s
76:	learn: 0.4851179	total: 446ms	remaining: 1.29s
77:	learn: 0.4848134	total: 452ms	remaining: 1.28s
78:	learn: 0.4842173	total: 457ms	remaining: 1.28s
79:	learn: 0.4832902	total: 463ms	remaining: 1.27s
80:	learn: 0.4829815	total: 468ms	remaining: 1.26s
81:	learn: 0.4824928	total: 473ms	remaining: 1.26s
82:	learn: 0.4819168	total: 479ms	remaining: 1.25s
83:	learn: 0.4811481	total: 484ms	remaining: 1.25s
84:	learn: 0.4805774	total: 490ms	remaining: 1.24s
85:	learn: 0.4789735	total: 495ms	remaining: 1.23s
86:	learn: 0.4781952	total: 501ms	remaining: 1.23s
87:	learn: 0.4772771	total: 506ms	remaining: 1.22s
88:	learn: 0.4765274	total: 512ms	remaining: 1.21s
89:	learn: 0.4762088	total: 517ms	r

...

2:	learn: 0.5742348	total: 1.05s	remaining: 1m 43s
3:	learn: 0.5478568	total: 1.45s	remaining: 1m 47s
4:	learn: 0.5287667	total: 1.8s	remaining: 1m 46s
5:	learn: 0.5111601	total: 2.16s	remaining: 1m 46s
6:	learn: 0.4999401	total: 2.5s	remaining: 1m 44s
7:	learn: 0.4889318	total: 2.8s	remaining: 1m 42s
8:	learn: 0.4829715	total: 3.11s	remaining: 1m 40s
9:	learn: 0.4775493	total: 3.41s	remaining: 1m 38s
10:	learn: 0.4647836	total: 3.73s	remaining: 1m 37s
11:	learn: 0.4570584	total: 4.1s	remaining: 1m 38s
12:	learn: 0.4489078	total: 4.46s	remaining: 1m 38s
13:	learn: 0.4420721	total: 4.79s	remaining: 1m 37s
14:	learn: 0.4382725	total: 5.12s	remaining: 1m 37s
15:	learn: 0.4337204	total: 5.44s	remaining: 1m 36s
16:	learn: 0.4305821	total: 5.79s	remaining: 1m 36s
17:	learn: 0.4284961	total: 6.2s	remaining: 1m 37s
18:	learn: 0.4265549	total: 6.58s	remaining: 1m 37s
19:	learn: 0.4248835	total: 6.95s	remaining: 1m 37s
20:	learn: 0.4220383	total: 7.33s	remaining: 1m 37s
21:	learn: 0.4201335	tota

161:	learn: 0.1444222	total: 56.2s	remaining: 47.9s
162:	learn: 0.1435296	total: 56.5s	remaining: 47.5s
163:	learn: 0.1431443	total: 56.8s	remaining: 47.1s
164:	learn: 0.1429680	total: 57.2s	remaining: 46.8s
165:	learn: 0.1404235	total: 57.5s	remaining: 46.4s
166:	learn: 0.1401221	total: 57.8s	remaining: 46s
167:	learn: 0.1391341	total: 58.1s	remaining: 45.7s
168:	learn: 0.1379602	total: 58.4s	remaining: 45.3s
169:	learn: 0.1378270	total: 58.7s	remaining: 44.9s
170:	learn: 0.1366664	total: 59.1s	remaining: 44.6s
171:	learn: 0.1347841	total: 59.4s	remaining: 44.2s
172:	learn: 0.1318497	total: 59.7s	remaining: 43.8s
173:	learn: 0.1312924	total: 1m	remaining: 43.5s
174:	learn: 0.1289795	total: 1m	remaining: 43.1s
175:	learn: 0.1285238	total: 1m	remaining: 42.8s
176:	learn: 0.1282206	total: 1m	remaining: 42.4s
177:	learn: 0.1255072	total: 1m 1s	remaining: 42s
178:	learn: 0.1244434	total: 1m 1s	remaining: 41.7s
179:	learn: 0.1243695	total: 1m 1s	remaining: 41.3s
180:	learn: 0.1227476	total:

41:	learn: 0.5131242	total: 325ms	remaining: 2s
42:	learn: 0.5124231	total: 334ms	remaining: 2s
43:	learn: 0.5113035	total: 345ms	remaining: 2.01s
44:	learn: 0.5107296	total: 356ms	remaining: 2.02s
45:	learn: 0.5093605	total: 364ms	remaining: 2.01s
46:	learn: 0.5084173	total: 377ms	remaining: 2.03s
47:	learn: 0.5072355	total: 389ms	remaining: 2.04s
48:	learn: 0.5056637	total: 399ms	remaining: 2.04s
49:	learn: 0.5045331	total: 411ms	remaining: 2.06s
50:	learn: 0.5040080	total: 422ms	remaining: 2.06s
51:	learn: 0.5027834	total: 437ms	remaining: 2.08s
52:	learn: 0.5014271	total: 449ms	remaining: 2.09s
53:	learn: 0.5012081	total: 461ms	remaining: 2.1s
54:	learn: 0.5009993	total: 474ms	remaining: 2.11s
55:	learn: 0.5004467	total: 484ms	remaining: 2.11s
56:	learn: 0.4988699	total: 496ms	remaining: 2.11s
57:	learn: 0.4982694	total: 510ms	remaining: 2.13s
58:	learn: 0.4967611	total: 522ms	remaining: 2.13s
59:	learn: 0.4961772	total: 535ms	remaining: 2.14s
60:	learn: 0.4955358	total: 546ms	rema

217:	learn: 0.4381222	total: 1.97s	remaining: 739ms
218:	learn: 0.4380335	total: 1.97s	remaining: 730ms
219:	learn: 0.4377987	total: 1.98s	remaining: 720ms
220:	learn: 0.4377831	total: 1.99s	remaining: 711ms
221:	learn: 0.4377567	total: 2s	remaining: 702ms
222:	learn: 0.4375135	total: 2s	remaining: 692ms
223:	learn: 0.4374558	total: 2.01s	remaining: 683ms
224:	learn: 0.4372133	total: 2.02s	remaining: 673ms
225:	learn: 0.4371260	total: 2.03s	remaining: 664ms
226:	learn: 0.4370159	total: 2.04s	remaining: 655ms
227:	learn: 0.4367198	total: 2.04s	remaining: 646ms
228:	learn: 0.4364523	total: 2.05s	remaining: 637ms
229:	learn: 0.4361792	total: 2.06s	remaining: 628ms
230:	learn: 0.4358610	total: 2.07s	remaining: 618ms
231:	learn: 0.4356377	total: 2.08s	remaining: 609ms
232:	learn: 0.4353132	total: 2.08s	remaining: 600ms
233:	learn: 0.4350573	total: 2.09s	remaining: 591ms
234:	learn: 0.4348098	total: 2.1s	remaining: 581ms
235:	learn: 0.4346312	total: 2.11s	remaining: 572ms
236:	learn: 0.43445

GridSearchCV(cv=3, error_score='raise',
       estimator=<catboost.core.CatBoostClassifier object at 0x7f93a5dc95c0>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'depth': [4, 7, 10], 'learning_rate': [0.03, 0.1, 0.15], 'l2_leaf_reg': [1, 4, 9], 'iterations': [300]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [12]:
pred = cb_model.predict(x_test)
accuracy_score(pred, y_test), 1-accuracy_score(pred, y_test)

(0.7857142857142857, 0.2142857142857143)

In [13]:
set_test = pd.merge(data,labels_predict,on = "person", how = "right")
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])

In [33]:
preds_kaggle = cb_model.predict_proba(set_test)

In [34]:
preds_kaggle  = preds_kaggle.tolist()
df_kaggel = pd.DataFrame(preds_kaggle)
df_kaggel.head()

,0,1
0,0.425870,0.574130
1,0.271243,0.728757
2,0.373263,0.626737
3,0.650986,0.349014
4,0.643535,0.356465


In [35]:
df_kaggel = pd.DataFrame(df_kaggel[1])                               
df_kaggel["person"] = personas
df_kaggel.rename(columns = {1: 'label'},inplace = True) 
df_kaggel = df_kaggel[['person','label']]

In [36]:
df_kaggel.head()

,person,label
0,00091926,0.574130
1,00091a7a,0.728757
2,000ba417,0.626737
3,000e4d9e,0.349014
4,000e619d,0.356465


In [37]:
df_kaggel["label"].value_counts()

0.056089    3
0.068350    3
0.064815    3
0.082682    2
0.060443    2
0.107483    2
0.095993    2
0.062316    2
0.040160    2
0.057981    2
0.056786    2
0.067395    2
0.056250    2
0.055068    2
0.066698    2
0.066559    2
0.063580    2
0.070153    2
0.063235    2
0.060698    2
0.058007    2
0.080667    2
0.057142    2
0.066252    2
0.057628    2
0.056016    2
0.057263    2
0.111446    2
0.066293    2
0.066954    2
           ..
0.141013    1
0.791159    1
0.734134    1
0.082294    1
0.802537    1
0.144944    1
0.757131    1
0.057603    1
0.265553    1
0.912411    1
0.133764    1
0.308414    1
0.461911    1
0.100068    1
0.839467    1
0.057402    1
0.762253    1
0.630920    1
0.191620    1
0.059147    1
0.344248    1
0.122413    1
0.539385    1
0.858945    1
0.439199    1
0.065179    1
0.260440    1
0.693165    1
0.090159    1
0.615518    1
Name: label, Length: 19382, dtype: int64

In [38]:
df_kaggel.to_csv('Resultados_CatBoost.csv',index=False)